In [49]:
import duckdb

con = duckdb.connect('tpch/dev.duckdb')
#con.sql("install tpch; load tpch;CALL dbgen(sf=0.01);")



In [50]:
con.sql('SELECT * FROM "dbt"."stg_part"')

┌──────────┬──────────────────────┬────────────────┬───┬────────────────┬───────────────────┬──────────────────────┐
│ part_key │      part_name       │   part_mrgr    │ … │ part_container │ part_retail_price │     part_comment     │
│  int32   │       varchar        │    varchar     │   │    varchar     │   decimal(15,2)   │       varchar        │
├──────────┼──────────────────────┼────────────────┼───┼────────────────┼───────────────────┼──────────────────────┤
│        1 │ goldenrod lavender…  │ Manufacturer#1 │ … │ JUMBO PKG      │            901.00 │ ironic pinto be      │
│        2 │ blush thistle blue…  │ Manufacturer#1 │ … │ LG CASE        │            902.00 │ nding, even pack     │
│        3 │ spring green yello…  │ Manufacturer#4 │ … │ WRAP CASE      │            903.00 │ ages. blithely fina  │
│        4 │ cornflower chocola…  │ Manufacturer#3 │ … │ MED DRUM       │            904.00 │ ckages. regul        │
│        5 │ forest brown coral…  │ Manufacturer#3 │ … │ SM PKG 

In [ ]:
con.close()

In [ ]:
con = duckdb.connect('tpch/dev.duckdb')
string = ''
for i in con.sql("SELECT table_name FROM  information_schema.tables").fetchall():
    string += f"drop table {i[0]};"
print(string)


In [ ]:
con.close()

In [ ]:
con = duckdb.connect('tpch/dev.duckdb')
con.sql("describe stg_orders")

In [ ]:
con.sql("describe stg_customer")

In [ ]:
con = duckdb.connect('tpch/dev.duckdb', read_only= True)
con.sql('SELECT * FROM stg_lineitem')

In [ ]:
query = '''
SELECT 
l_orderkey as lineitem_order_key, 
l_partkey as lineitem_part_key, 
l_suppkey as lineitem_supplied_key,
l_linenumber as lineitem_linenumber,
---- I'm gonna assume that any value that isn't 'N' is a return 
case when l_returnflag = 'N' then l_quantity else 0 - l_quantity end as lineitem_quantity,
l_extendedprice as lineitem_extended_price,
l_discount as lineitem_discount, 
l_tax as lineitem_tax, 
(l_extendedprice - l_discount) * (case when l_returnflag = 'N' then l_quantity else 0 - l_quantity end) as lineitem_line_total,
((l_extendedprice - l_discount) * (1 + l_tax)) * (case when l_returnflag = 'N' then l_quantity else 0 - l_quantity end) as lineitem_line_total_tax,
l_returnflag as lineitem_return_flag,
l_linestatus as lineitem_line_status,
l_shipdate as lineitem_ship_date, 
l_commitdate as lineitem_commit_date, 
l_receiptdate as lineitem_receipt_date,
l_shipinstruct as lineitem_ship_instruct,
l_shipmode as lineitem_ship_mode,
l_comment as lineitem_comment
from lineitem
'''

con.sql(query)

In [ ]:
#con = duckdb.connect('tpch/dev.duckdb', read_only= True)
table = 'region'
cols = con.query(f'''
------SELECT '{table}.' || column_name || ',' FROM information_schema.columns where table_name = '{table}'
SELECT column_name || ',' FROM information_schema.columns where table_name = '{table}'
''').fetchall()

print('SELECT')
for i in cols:
    
    print(i[0])

In [ ]:
con.close()

In [ ]:
con.sql('SELECT * FROM order_info_wide ')

In [ ]:
from duckdb import sql 
sql("SELECT * FROM read_parquet('tpch/order_info_wide.parquet')")